## Отчет по штатному расписанию <img style="float: right;" src="../00_system/VTB.png">

Отражение ситуации на конец отчетного месяца по сотрудникам: штатная численность, фактическая численность, новые и ушедшие сотрудники, вакансии

> **Частота обновления:**<br>
> * ежемесячно, ~ до 5 числа следующим за отчетным <br>

> **Ответственные:** <br> 
> * за разработку: ФИО <name@.ru>, ФИО <name@.ru> <br> 
> * за данные: ФИО <name@.ru> <br>
   
> **Ссылки на отчеты:** <br>
> * дашборд `Excel` - `\\*\Штатное расписание ЦАП`
> * дашборд `Qlik` -  https://.ru

> **Ссылки на исходные файлы:** <br>
> * справочники: `\\*\dics` <br>
> * исходные файлы: `\\*\raw`

### Импортируем библиотеки

In [1]:
#увеличиваем рабочую область
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd #работа с таблицами
import os #для работы с папками
pd.options.mode.chained_assignment = None #Отключение предупреждения Pandas "SettingWithCopyWarning:A value is trying to be set on a copy of a slice from a DataFrame"
import datetime #для сохранения в имени файла даты и времени сохранения
import numpy as np
import msoffcrypto #для работы по расшифровке книги по паролю
import io #для работы по расшифровке книги по паролю

### Функции для работы с файлами

#### Функция для открытия файлов

In [3]:
# Функция для открытия файла и передачи для дальнейшей обработки
#  Переменные на вход:
#   - file_ - путь к файлу
#   - file_mask - маска наименовании файла
#   - need_split - требуется ли разделение названия файла, используется по файлам штатной численности, по умолчанию False
#   - sheet_name_ - None если все листы, 0 - первый лист, остальное - если конкретное
#   - header_ - строка с заголовками
#   - skip_rows - пропуск строк сверху
#   - usecols_ - определенные столбцы к загрузке
#   - password_ - пароль к книге Excel при необходимости
#
def open_file (file_, file_mask, need_split = False, sheet_name_ = None, header_ = 0, skip_rows = None, usecols_ = None, password_ = None):
    #определение, файл ли это
    is_file = os.path.isfile(file_)
    #определить расширение файла
    file_ext = os.path.splitext(file_)[1].lower()
    #определяем движок в зависимости от типа файла
    engine_ = 'xlrd' if file_ext == '.xls' else 'openpyxl'
    #определяем наименование файла
    file_name = os.path.basename(file_)
    #если это файл, движок определен и название файла попадает под маску - осуществляем перебор
    if is_file and file_mask in file_name:
        #открываем файл
        with open(file_,'rb') as f:
            #если у нас Excel файл с паролем, то перед открытием мы его снимаем, password_ должен быть подан на вход
            if password_ is not None:
                file_ = io.BytesIO()
                file = msoffcrypto.OfficeFile(f)
                file.load_key(password = password_)  # Use password
                file.decrypt(file_)
            else:
                file_ = f #если
            #открываем книгу Excel
            try:
                df = pd.read_excel(file_, 
                    sheet_name= sheet_name_,
                    engine = engine_, 
                    header= header_,
                    skiprows = skip_rows,
                    usecols = usecols_,
                    index_col = None)
                #файл источник
                df['Дата_отчета'] = '01-' + file_name.split("_")[2][:-5] if need_split else file_name
                #отбивка об обработке
                print(f"Обработан файл - '{file_name}")
                #возвращаем датафрем
                return df
            except Exception as e:
                print(f"Не удалось обработать файл - '{file_name}', с ошибкой - {e}")

#### Функция для сборки файлов по штатной численности

In [4]:
def merging_staff_files(dir_in):
    #наименование функции или блока для отработки ошибок
    unit = 'merging_staff_files'
    #dir_in - расположение файлов прогнозов
    #dir_out - куда сохраняем итоговые файл
    #file_name - наименование итогового файла.
    try:
        #загружаемые столбцы
        heads = ['ID Штатной единицы','Unnamed: 5','Unnamed: 6','Unnamed: 7','Unnamed: 8',
                 'Должность','Штатная численность','Фактическая численность','ФИО','Таб. №','Город, ФН','Признак ШЕ',
                 'Дата начала действия признака','Дата окончания отпуска по уходу за ребенком','Дата увольнения',
                 'Примечание']
        #создаем итоговую таблицу
        df_total = pd.DataFrame()
        #перебираем файлы в папке
        for entry in os.scandir(dir_in): #перебираем все файлы в папке
            #открываем каждый файл в папке
            df = open_file (entry.path, 'staff_regions', need_split = True, sheet_name_ = 0, skip_rows = 5, usecols_ = heads)
            #продолжить, если таблица пустая
            if df is not None:
                #временная таблица
                df_temp = pd.DataFrame()
                #удаляем две лишние строки т.к. заголовок был многоуровневый
                df_temp = df.iloc[2:,:]
                #переименование столбцов
                df_temp.columns = ['id','Управление','ЦАП','ПАП','Отдел','Должность','ШЧ','ФЧ','ФИО','ТН','Город','Признак_ШЕ','Дата_начала_действия_признака','Дата_окончания_отпуска_по_уходу_за_ребенком','Дата_увольнения','Примечание','Дата_отчета']
                #переносим итоговый результат в итоговую таблицу
                df_total = pd.concat([df_total, df_temp])
                #очищаем временную таблицу
                df_temp = None
            #очищаем df
            df = None
        #возвращаем обработанные данные
        return df_total
    except Exception as e:
        err_descr (unit, e)

#### Функция для сборки файлов по движению персонала

In [5]:
def merging_staff_moving_files(dir_in):
    #наименование функции или блока для отработки ошибок
    unit = 'merging_staff_moving_files'
    #dir_in - расположение файлов прогнозов
    #dir_out - куда сохраняем итоговые файл
    #file_name - наименование итогового файла.
    try:
        #загружаемые столбцы
        heads = ['Таб. №','Дата приема']
        #создаем итоговую таблицу
        df_total = pd.DataFrame()
        for entry in os.scandir(dir_in): #перебираем все файлы в папке
            #открываем каждый файл в папке
            df = open_file (entry.path, 'Отчет по движению персонала', need_split = False, sheet_name_ = 0, skip_rows = 3, usecols_ = heads, password_ = '2023')
            #продолжить, если таблица пустая
            if df is not None:
                #временная таблица
                df_temp = pd.DataFrame()
                #удаляем две лишние строки т.к. заголовок был многоуровневый
                df_temp = df.iloc[2:,:]
                #переименование столбцов
                #df_temp.columns = ['id','Управление','ЦАП','ПАП','Отдел','Должность','ШЧ','ФЧ','ФИО','ТН','Город','Признак_ШЕ','Дата_начала_действия_признака','Дата_окончания_отпуска_по_уходу_за_ребенком','Дата_увольнения','Примечание','Дата_отчета']
                #переносим итоговый результат в итоговую таблицу
                df_total = pd.concat([df_total, df_temp])
                #очищаем временную таблицу
                df_temp = None
        #очищаем df
        df = None
        #возвращаем обработанные данные
        return df_total
    except Exception as e:
        err_descr (unit, e)

#### Функция для возвращения текста ошибки

In [6]:
def err_descr(unit_, exception_text):
    print(f"Ошибка: блок/функция '{unit_}', {exception_text}")

#### Функция для возвращения успешного статуса

In [7]:
def if_success(unit_):
    print(f"Выполнено: блок/функция '{unit_}'")

### Предобработка данных

#### Загрузка данных
Объединение исходных данных в одну таблицу

In [8]:
#Местоположение загрузочных файлов
file_strg = r'\\*\Данные'
#Объединение всех файлов в папке в одну таблицу
df_staff = merging_staff_files (file_strg + "\\raw")

Обработан файл - 'staff_regions_01-2022.xlsm
Обработан файл - 'staff_regions_01-2023.xlsm
Обработан файл - 'staff_regions_02-2022.xlsm
Обработан файл - 'staff_regions_02-2023.xlsm
Обработан файл - 'staff_regions_03-2022.xlsm
Обработан файл - 'staff_regions_03-2023.xlsm
Обработан файл - 'staff_regions_04-2022.xlsm
Обработан файл - 'staff_regions_04-2023.xlsx
Обработан файл - 'staff_regions_05-2022.xlsm
Обработан файл - 'staff_regions_06-2022.xlsm
Обработан файл - 'staff_regions_07-2022.xlsm
Обработан файл - 'staff_regions_08-2022.xlsm
Обработан файл - 'staff_regions_09-2022.xlsm
Обработан файл - 'staff_regions_10-2022.xlsm
Обработан файл - 'staff_regions_11-2022.xlsm
Обработан файл - 'staff_regions_12-2022.xlsm


Объединение данных по уволенным сотрудникам в одну таблицу

In [9]:
# ! отключил т.к. уволенные проверяются сверкой ТН из предыдущего месяца в текущем
#file_strg_m = r'\\*\Движение персонала ЦАП'
#df_staff_m = merging_staff_moving_files (file_strg_m + "\\2023")

### Предварительная обработка

#### Временная таблица для обработки
Создаем временную таблицу для дальнейшей обработки данных

In [10]:
# cоздаем временную таблицу для обработки
df_t = df_staff.copy()

#### Дополнительные столбцы по статичному меппингу
Накладываем меппинг, который находится в папках со справочниками 

In [11]:
#наименование функции или блока для отработки ошибок
unit = 'Дополнительные столбцы по статичному меппингу'
try:
    #открываем файл меппинга
    df_mapping = open_file (file_strg + '\\dics\dics_set.xlsx', 'dics', False)

    #----------------------------------------------------------------------------------------------
    # 1. Накладываем категории должностей
    df_t = df_t.merge(df_mapping['positions_cat'], on = 'Должность', how = 'left')
    #проверка на отсутствующие аналитики
    #--посчитаем количество строк с отсутствующими аналитиками
    check_location = df_t.query("Категория_должности.isnull()").shape[0]
    #--если есть ненайденные, то вывод строк, которые не нашлись в меппинге
    if check_location > 0:
        print('Не найдены следующие аналитики по категориям должностей', display(df_t.query("Категория_должности.isnull()")))

    #----------------------------------------------------------------------------------------------
    # 2. Накладываем ПАП-ы и ЦАП-ы отталкиваясь от ПАП
    #--для начала объединим столбец ПАП и ЦАП т.к. ПАП не везде заполнен, если ПАП пустой, заполняем из ЦАП
    df_t.loc[df_t['ПАП'].isnull(),'ПАП'] = df_t.loc[:,'ЦАП']
    #--применяем меппинг по локациям
    df_t = df_t.merge(df_mapping['locations'], left_on = 'ПАП', right_on = 'ПАП_полный', how = 'left')
    #--проверка на отсутствующие аналитики
    #--посчитаем количество строк с отсутствующими аналитиками
    check_location = df_t.query("ПАП_полный.isnull()").shape[0]
    #--если есть ненайденные, то вывод строк, которые не нашлись в меппинге
    if check_location > 0:
        print('Не найдены следующие аналитики по определению ПАП', display(df_t.query("ПАП_полный.isnull()")))

    #----------------------------------------------------------------------------------------------
    # 3. Накладываем корректировки по техническим вакансиям
    df_t = df_t.merge(df_mapping['corr_tech'], on = 'id', how = 'left')
    #--добавляем в столбец начальной даты, где не было заполнено, дату минимальную для нашего отчета 2021-01-01, чтобы корректно определять диапазон, когда ТН был техническим
    df_t.loc[df_t.tech_date_end.notnull()&df_t.tech_date_st.isnull(),'tech_date_st'] = datetime.date(2021,1,1)
    #----------------------------------------------------------------------------------------------
    # Конвертируем столбцы с датами в формат дат для корректного сравнения
    #--столбец Дата_отчета, месяца из наименования файлов, конвертируем в формат, использующийся во всей таблице
    df_t[df_t.columns[16]] = pd.to_datetime(df_t[df_t.columns[16]], format = "%d-%m-%Y")
    #--меняем столбцы с датами на единый формат даты
    cols_d = np.r_[12:15,21:23]
    df_t[df_t.columns[cols_d]] = df_t.loc[:,df_t.columns[cols_d]].apply(pd.to_datetime, format = "%d.%m.%Y")
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Обработан файл - 'dics_set.xlsx
Выполнено: блок/функция 'Дополнительные столбцы по статичному меппингу'


#### Дополнительные столбцы по условиям таблицы
Создаем столбцы с дополнительными признаками

In [12]:
#наименование функции или блока для отработки ошибок
unit = 'Дополнительные столбцы по статичному меппингу'
try:
    #Вакансия
    df_t.loc[df_t['Признак_ШЕ'].str.contains('ВАК', na = False),'Ставка_вак'] = 1
    #Декретная ставка
    df_t.loc[df_t['Признак_ШЕ'].str.contains('ДЕК', na = False),'Ставка_дек'] = 1
    #Замещенная ставка
    df_t.loc[df_t['Признак_ШЕ'].str.contains('ЗАМ', na = False),'Ставка_зам'] = 1
    #Техническая вакансия
    df_t.loc[df_t.query('tech_date_st <= Дата_отчета <= tech_date_end').index, 'Ставка_тех'] = 1
    #--удаляем вспомогательные столбцы с датами начала действия и окончания технической вакансии, которая была подтянута из справочника
    df_t = df_t.drop(columns=['tech_date_st','tech_date_end'])
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Дополнительные столбцы по статичному меппингу'


#### Изменение типов данных

In [13]:
#наименование функции или блока для отработки ошибок
unit = 'Дополнительные столбцы по статичному меппингу'
try:
    #изменение текущих столбцов с float to int
    #--столбцы к конвертации
    flt_int_cols = ['id','ТН','Ставка_вак','Ставка_дек','Ставка_зам','Ставка_тех']
    #--конвертация
    df_t[flt_int_cols] = df_t[flt_int_cols].fillna(0).astype('int')
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Дополнительные столбцы по статичному меппингу'


#### Дополнительные столбцы по виртуальным меппингам
Применяем меппинги, которые были созданы во время обработки выгрузки

* Увольняющиеся сотрудники по дате увольнения

In [14]:
#наименование функции или блока для отработки ошибок
unit = 'Увольняющиеся сотрудники по дате увольнения'
try:
    #формируем отдельную таблицу по увольняемым сотрудникам, у которых проставлены даты увольнения
    df_fired = df_t.groupby(['id','Дата_увольнения']).agg(Количество = ('id','count')).iloc[:,:-1].reset_index()
    #прибавляем 1 день к дате увольнения т.к. если стоит последний день, то сотрудник ещё числится в штате 
    df_fired.loc[:,'Дата_увольнения'] += pd.DateOffset(days=1)
    #добавляем признак уволенного сотрудника
    df_t = df_t.merge(df_fired,
               left_on = [df_t.id, df_t.Дата_отчета.dt.year, df_t.Дата_отчета.dt.month], 
               right_on = [df_fired.id, df_fired.Дата_увольнения.dt.year, df_fired.Дата_увольнения.dt.month], 
               how = "left", suffixes = ['','_факт']).iloc[:,3:].drop(columns = ['id_факт'])
    #очистка переменной временной таблицы
    df_fired = None
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Увольняющиеся сотрудники по дате увольнения'


* Длительность поиска сотрудника на вакансию

In [15]:
#наименование функции или блока для отработки ошибок
unit = 'Длительность поиска сотрудника на вакансию'
try:
    #создаем справочник, в котором будут только штатные единицы с вакансиями
    df_vac = df_t[df_t['Ставка_вак'] == 1].groupby(['id','Дата_отчета']).agg(Количество = ('id','count')).reset_index()
    #--создаем столбец отклонения последовательности, в котором сравниваются даты в рамках одной группы "id" по последовательности дат, если месяц за месяцем, то расчитывается количество дней
    df_vac.loc[:,'Отклонение_последовательности'] = df_vac.groupby('id')[['id','Дата_отчета']].diff(1)['Дата_отчета']
    #--по отклонениям свыше 31 дня ставим NaN т.к. мы ищем только отклонение Month-to-Month
    df_vac.loc[df_vac['Отклонение_последовательности'] > pd.Timedelta('31 days'),'Отклонение_последовательности'] = None
    #--ставим дату как в источнике, если отклонение пустое, это означает, что это стартовая дата т.к. перед ней нет сравнительных дат
    df_vac.loc[df_vac['Отклонение_последовательности'].isnull(),'Дата_начала_последовательности'] = df_vac['Дата_отчета']
    #--меняем формат данных id на int, для того чтобы в функции merge_asof корректно определялся параметр by
    df_vac['id'] = df_vac['id'].astype('int')
    #итоговая таблица с меппингом
    #--создаем столбец, от которого будем рассчитывать протяженность последовательного наличия вакантной ставки с помощью merge_asof, суть в том, что к ID из левой таблицы будут браться даты ближайшие от правой таблицы и мы поймем, сколько ставка у нас находится в поиске
    df_vac = pd.merge_asof(df_vac.sort_values(by=['Дата_отчета','id']),
                           df_vac[['id','Дата_начала_последовательности']].query("Дата_начала_последовательности.notnull()").sort_values(by=['Дата_начала_последовательности','id']), 
                           left_on='Дата_отчета', 
                           right_on = 'Дата_начала_последовательности', 
                           by = 'id',
                           suffixes=('','_расчет'),
                           direction='backward')
    #рассчитываем длительность поиска по вакансии и прибавляем 1 день т.к. отклонение месяц в месяц будет нулем
    df_vac['Вакансия_длительность_поиска'] = (df_vac['Дата_отчета'].dt.to_period('M').astype('int64') - df_vac['Дата_начала_последовательности_расчет'].dt.to_period('M').astype('int64')) + 1
    #убираем лишние столбцы
    df_vac = df_vac.iloc[:,[0,1,-1]]
    #применяем соединиение с основной таблицей
    df_t = df_t.merge(df_vac,
               on = ['id', 'Дата_отчета'], 
               how = "left")
    #очистка переменной временной таблицы
    df_vac = None
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Длительность поиска сотрудника на вакансию'


* Перемещения внутри ID<br>
 *В данной проверке мы проверяем помесячно по каждой ID, были ли какие-либо действия внутри ID в части табельного номера, если да, будет проставлен признак `1`*

In [16]:
#наименование функции или блока для отработки ошибок
unit = 'Перемещения внутри ID'
try:
    #СОЗДАНИЕ МЕППИНГА ID - Дата отчета - ТН
    #--создаем таблицу с ID, датами отчетов и объединенными ТН, в случае если на ID несколько ТН
    df_movement = df_t.groupby(['id','Дата_отчета'])['ТН'].apply(list).reset_index()
    #--создаем столбец со сдвигом в +1 месяц, чтобы сравнить текущие данные с предыдущим месяцем т.е. в сдвинутой дате не будет предыдущего месяца
    df_movement['Дата_отчета_сдвиг'] = df_movement['Дата_отчета'] + pd.DateOffset(months=1)
    #--путем соединения текущей и сдвинутой даты, показываем, какой ТН был в прошлом месяце
    df_movement = df_movement.merge(df_movement, 
                                      left_on = ['id','Дата_отчета'], 
                                      right_on = ['id','Дата_отчета_сдвиг'], 
                                      how = 'left', 
                                      suffixes = ['','_пред']).drop(columns=['Дата_отчета_пред','Дата_отчета_сдвиг_пред'])
    #--добавляем столбец минимальной даты по id, чтобы исключить ненайденные значения т.к. проверять менее минимальной невозможно
    df_movement = df_movement.merge(df_t.groupby(['id']).agg(Дата_мин=('Дата_отчета',np.min)).reset_index(),
                                      left_on = ['id','Дата_отчета'], 
                                      right_on = ['id','Дата_мин'], 
                                      how = 'left')
    #--если дата отчета совпадает с минимальной датой, то ставим ТН_пред равный ТН т.к. предыдущих данных нет для сравнения
    df_movement.loc[df_movement['Дата_отчета'] == df_movement['Дата_мин'], 'ТН_пред'] = df_movement['ТН']
    #--осуществляем проверку, если ТН и ТН_пред совпадает, значит по ID штатной единицы изменений не было, если нет, то значит произошло изменение
    df_movement.loc[df_movement['ТН'] != df_movement['ТН_пред'], 'ID_изменение'] = 1
    #--оставляем только те ID, по которым происходили изменения
    df_movement = df_movement.query("ID_изменение > 0")
    #ОБЪЕДИНЕНИЕ С ИТОГОВОЙ ТАБЛИЦЕЙ
    #применяем соединиение с основной таблицей
    df_t = df_t.merge(df_movement[['id','Дата_отчета','ID_изменение']],
                      on = ['id', 'Дата_отчета'], 
                      how = "left", 
                      suffixes = ['','_пред'])
    #очистка переменной временной таблицы
    df_movement = None
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Перемещения внутри ID'


* Новые и ушедшие ТН<br>
 *В данной проверке мы проводим сверку по всем табельным номерам: 1) если в предыдущем месяце нет, значит ТН новый 2) если из прошлого месяца сотрудника нет, значит такого ТН больше нет*

In [17]:
#наименование функции или блока для отработки ошибок
unit = 'Новые и ушедшие ТН'
try:
    #СОЗДАНИЕ МЕППИНГА Дата отчета - ТН
    #--создаем таблицу с датами отчетов и ТН
    df_retired_new = df_t.query('ТН != 0').groupby(['ТН','Дата_отчета']).agg(Количество=('ТН','count')).reset_index()
    #--создаем столбец со сдвигом даты отчета в +1 месяц, чтобы сравнить текущие данные с предыдущим месяцем и наоборот т.е. в сдвинутой дате не будет предыдущего месяца
    df_retired_new['Дата_отчета_сдвиг'] = df_retired_new['Дата_отчета'] + pd.DateOffset(months=1)
    #--путем соединения текущей и сдвинутой даты, показываем, какой ТН был в прошлом месяце
    df_retired_new = df_retired_new.merge(df_retired_new, 
                                      left_on = ['ТН','Дата_отчета'], 
                                      right_on = ['ТН','Дата_отчета_сдвиг'], 
                                      how = 'left', 
                                      suffixes = ['','_пред']).drop(columns=['Дата_отчета_пред','Дата_отчета_сдвиг_пред'])
    #--путем соединения текущей и сдвинутой даты, показываем, какой ТН присутствует в следующем месяце
    df_retired_new = df_retired_new.merge(df_retired_new, 
                                      left_on = ['ТН','Дата_отчета_сдвиг'], 
                                      right_on = ['ТН','Дата_отчета'], 
                                      how = 'left', 
                                      suffixes = ['','_след']).drop(columns=['Количество_пред_след','Дата_отчета_след','Дата_отчета_сдвиг_след'])
    #--если дата отчета совпадает с минимальной/максимальной датой, то ставим Количество_пред/Количество_след соответственно равный столбцу Количество т.к. предыдущих/будущих данных нет для сравнения
    df_retired_new.loc[df_retired_new['Дата_отчета'] == df_retired_new['Дата_отчета'].min(), 'Количество_пред'] = 1
    df_retired_new.loc[df_retired_new['Дата_отчета'] == df_retired_new['Дата_отчета'].max(), 'Количество_след'] = 1
    #--меняем признаки, 1 в 0, NaN в 1 т.к. именно NaN показывал, что значение не было найдено
    df_retired_new.iloc[:,4:] = (df_retired_new.iloc[:,4:] - 1).fillna(1)
    #--переименуем последние столбцы
    df_retired_new.rename({'Количество_пред': 'ТН_новый', 'Количество_след': 'ТН_ушедший'}, axis=1, inplace=True)
    #ОБЪЕДИНЕНИЕ С ИТОГОВОЙ ТАБЛИЦЕЙ
    #--применяем соединиение с основной таблицей
    df_t = df_t.merge(df_retired_new[['ТН','Дата_отчета','ТН_новый','ТН_ушедший']],
                      on = ['ТН', 'Дата_отчета'], 
                      how = "left", 
                      suffixes = ['','_пред'])
    #--добавляем новый столбец с датой отчета со сдвигом на 1 месяц вперед, чтобы в отчетности видеть к какому месяцу относится увольнения т.к. мы сверяли предыдущий с текущим и проставляли признак уволенного в предыдущем, но по факту сотрудник ещё был в этом месяце
    df_t['Дата_отчета_сдвиг'] = df_t['Дата_отчета'] + pd.DateOffset(months=1)
    #очистка переменной таблицы меппинга
    df_retired_new = None
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Новые и ушедшие ТН'


### Сохранение результата

Определяем последнюю дату по загруженным данным и вставим её в название файла

In [18]:
report_date = str(df_t['Дата_отчета'].max().date())

Сохраняем таблицу в файл Excel

In [19]:
unit = 'Сохранение данных'
try:
    df_t.to_excel(file_strg + "\\reports\staff_regions_report_" + report_date + ".xlsx", sheet_name = 'Данные', index = False)
    #статус по завершению обработки блока/функции
    if_success(unit)
except Exception as e:
    err_descr (unit, e)

Выполнено: блок/функция 'Сохранение данных'


Очистка переменных с основными таблицами

In [20]:
#таблица исходная при загрузке
#df_staff = None
#копия таблицы для обработки
#df_t = None